In [1]:
import pandas as pd 
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df_train = pd.read_csv('../static/data/pima_train.csv')
df_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,101.0,50.000000,15.000000,36.000000,24.2,0.526,26,0
1,0,141.0,69.105469,20.536458,79.799479,42.4,0.205,29,1
2,0,118.0,84.000000,47.000000,230.000000,45.8,0.551,31,1
3,5,147.0,78.000000,20.536458,79.799479,33.7,0.218,65,0
4,10,122.0,68.000000,20.536458,79.799479,31.2,0.258,41,0


In [3]:
X_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values
X_train.shape, y_train.shape

((576, 8), (576,))

In [4]:
df_test = pd.read_csv('../static/data/pima_test.csv')
df_test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,7,133.0,84.0,20.536458,79.799479,40.2,0.696,37,0
1,2,90.0,70.0,17.000000,79.799479,27.3,0.085,22,0
2,8,194.0,80.0,20.536458,79.799479,26.1,0.551,67,0
3,1,83.0,68.0,20.536458,79.799479,18.2,0.624,27,0
4,6,125.0,68.0,30.000000,120.000000,30.0,0.464,32,0


In [5]:
X_test = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values
X_test.shape, y_test.shape

((192, 8), (192,))

In [6]:
scaler = StandardScaler()
scaler.fit(df_train.iloc[:, :-1])

StandardScaler()

In [7]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((576, 8), (192, 8))

In [8]:
joblib.dump(scaler, '../static/model/pima_scaler.pkl')

['../static/model/pima_scaler.pkl']

### 1. Logistic Regression

In [9]:
lr_clf = LogisticRegression()
lr_clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [15]:
params = {
    # 'C': [1, 3, 5, 7, 10, 20]
    'C': [0.01, 0.1, 0.2, 0.3, 1, 2, 3, 4, 5 ]
}

In [16]:
grid_cv = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.7605
최적 파라미터: {'C': 3}


In [17]:
best_lr = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_lr.predict(X_test_scaled)
acc_lr = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_lr:.4f}')

평균 정확도 : 0.8021


In [18]:
joblib.dump(best_lr, '../static/model/pima_lr.pkl')

['../static/model/pima_lr.pkl']

### 2.Decision Tree

In [19]:
dt_clf = DecisionTreeClassifier()
dt_clf.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [20]:
params = {
    'max_depth': [0.9, 1, 2, 5, 8, 10],
    'min_samples_split': [1, 2, 3, 4]
}

In [21]:
grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.7136
최적 파라미터: {'max_depth': 1, 'min_samples_split': 2}


In [22]:
best_dt = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_dt.predict(X_test_scaled)
acc_dt = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_dt:.4f}')

평균 정확도 : 0.7083


In [23]:
joblib.dump(best_dt, '../static/model/pima_dt.pkl')

['../static/model/pima_dt.pkl']

### 3. SVM

In [24]:
sv_clf = SVC()
sv_clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [33]:
params = {
    # 'C': [90, 91, 92, 93, 94, 95]
     'C': [ 1, 2, 3, 4, 5, 7, 9, 10]
}

In [34]:
grid_cv = GridSearchCV(sv_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.7484
최적 파라미터: {'C': 3}


In [35]:
best_sv = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_sv.predict(X_test_scaled)
acc_sv = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_sv:.4f}')

평균 정확도 : 0.7448


In [36]:
joblib.dump(best_sv, '../static/model/pima_sv.pkl')

['../static/model/pima_sv.pkl']

### 4.랜덤 포레스트(Random Forest)

In [37]:
rf_clf = RandomForestClassifier()
rf_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [43]:
params = {
    'max_depth': [1, 2, 3, 4, 6, 7, 8, 9, 10],
    'min_samples_split': [1, 2, 3, 4, 5, 6, 10]
}

In [44]:
grid_cv = GridSearchCV(rf_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.7553
최적 파라미터: {'max_depth': 6, 'min_samples_split': 5}


In [45]:
best_rf = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_rf.predict(X_test_scaled)
acc_rf = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_rf:.4f}')

평균 정확도 : 0.7917


In [46]:
joblib.dump(best_rf, '../static/model/pima_rf.pkl')

['../static/model/pima_rf.pkl']

### 5.K 최근접 이웃(K Nearest Neighbor)

In [47]:
kn_clf = KNeighborsClassifier()
kn_clf.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [52]:
params = {
    'n_neighbors': [10, 12, 13, 14, 15, 17, 19, 20]
}

In [53]:
grid_cv = GridSearchCV(kn_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.7570
최적 파라미터: {'n_neighbors': 14}


In [54]:
best_kn = grid_cv.best_estimator_ # best_estimator_ : 최적의 파라미터로 모델 생성
pred = best_kn.predict(X_test_scaled)
acc_kn = accuracy_score(y_test, pred)
print(f'평균 정확도 : {acc_kn:.4f}')

평균 정확도 : 0.7552


In [55]:
joblib.dump(best_kn, '../static/model/pima_kn.pkl')

['../static/model/pima_kn.pkl']

### Test

In [56]:
index = 50

In [57]:
test_data = (df_test.iloc[index, :-1].values).reshape(1,-1)
label = df_test.iloc[index, -1]
test_data, label

(array([[  7.   , 150.   ,  78.   ,  29.   , 126.   ,  35.2  ,   0.692,
          54.   ]]),
 1)

In [58]:
new_scaler = joblib.load('../static/model/pima_scaler.pkl')

In [59]:
test_scaled = new_scaler.transform(test_data)

In [60]:
pred_lr = best_lr.predict(test_scaled)
pred_sv = best_sv.predict(test_scaled)
pred_dt = best_dt.predict(test_scaled)
pred_rf = best_rf.predict(test_scaled)
pred_kn = best_kn.predict(test_scaled)

In [61]:
label, pred_lr[0], pred_sv[0], pred_dt[0], pred_rf[0], pred_kn[0]

(1, 1, 1, 1, 1, 1)

In [62]:
pima_dict = {'label':label, 'pred_lr':pred_lr[0], 'area_ratio': pred_sv[0], 'per_person':pred_dt[0], 'pred_rf': pred_rf[0], 'pred_kn': pred_kn[0]}
pima_dict

{'label': 1,
 'pred_lr': 1,
 'area_ratio': 1,
 'per_person': 1,
 'pred_rf': 1,
 'pred_kn': 1}

In [63]:
a = dict(zip(df_test.columns[:-1], df_test.iloc[index, :-1]))
a

{'Pregnancies': 7.0,
 'Glucose': 150.0,
 'BloodPressure': 78.0,
 'SkinThickness': 29.0,
 'Insulin': 126.0,
 'BMI': 35.2,
 'DiabetesPedigreeFunction': 0.6920000000000001,
 'Age': 54.0}